In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## IMPORTS
import torch
from torch.nn.utils.rnn import pad_sequence
from torch import nn, Tensor,multinomial
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, Transformer
from torch.utils.data import dataset, DataLoader
from torch.autograd import Variable
from torchtext.vocab import Vectors, vocab,build_vocab_from_iterator,vocab

import math
import re
import pickle
from collections import defaultdict, OrderedDict, Counter
from tqdm import tqdm
import numpy as np
from numpy.random import uniform
import os

In [3]:
!pip -q install transformers evaluate peft python-dotenv huggingface_hub wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
## CONSTANTS
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lang2code = {
    "Gitksan" : "git", 
    "Arapaho" : "arp", 
    "Lezgi" : "lez", 
    "Nyangbo" : "nyb", 
    "Tsez" : "ddo",
    "Uspanteko" : "usp",  
    "Natugu" : "ntu"
    }

code2lang = {v:k for k,v in lang2code.items()}

START = "<start>"
END = "<end>"
UNK = "<unk>"
PAD = "<pad>"
SP = "#"
SPECIALS = [PAD, START, END,UNK,SP, "?","!", "1","2","3","4","5","6","7","8","9","0"]

PATH = "/content/drive/MyDrive/Colab_Notebooks/2023glossingST/" 

LOAD_FROM_CKPT = False


In [ ]:
# from dotenv import load_dotenv
# load_dotenv("/content/drive/MyDrive/Colab_Notebooks/.env")
# HF = os.getenv("HF_TOKEN")
# WANDB = os.getenv("WANDB_TOKEN")

# !huggingface-cli login --token $HF
# !wandb login $WANDB

# Preprocessing

## Helpers

In [7]:

def tag_bd(seq:list):
  '''
  Attaches boundary tokens to previous and following characters in tokenized gloss w/ character-level tokenization
  to preserve delimiters.

  E.g. ["I", "N" - "s"] -> ["I", "N-", "-s"]
  '''
  BDS = {"-", "."}
  for i, c in enumerate(seq):
    if i < len(seq)-2:
      if seq[i+1] in BDS:
          seq[i] += seq[i+1]
          seq[i+2] = seq[i+1] + seq[i+2]
  return [i for i in seq if not i in BDS]

# Preprocessing pipelines 
orth_process = lambda x: [START] + list(re.sub(" ", SP, x)) + [END]              # source (transcription) line
gloss_process = lambda x: [START] + tag_bd(list(re.sub(" ", SP, x))) + [END]     # target (gloss) line


def get_linesplits(orths:list, 
                   glosses:list):
  ex = [(orth_process(o), gloss_process(g)) for o, g in zip(orths, glosses)]
  return ex

## Simple segmentation-based augmentation

In [8]:
def get_windows(glosses:list,prop=0.7):
    '''
    Randomly samples window sizes from uniform distribution 
    upper length thresholds based on training gloss lengths, modulated by parameter `prop`
    lower length threshold = 1, to ensure word-level representation 
    '''
    counts=defaultdict(int) # Count frequency of gloss lengths 

    for gloss in glosses:
        counts[len(gloss.split())] += 1
    
    sorted_counts = sorted(counts.items(),key=lambda x:x[1],reverse=True)

    lcounts = [l for l,c in sorted_counts]
    avg = round(sum(lcounts)/len(lcounts)) * prop
    
    while True:
        windows = np.random.randint(1,avg,int(avg))
        if len(set(windows)) == int(avg)-1:
            break
    return set(windows)

def get_window_segs(transcriptions:list, glosses:list, windows:list):
    '''
    Generate synthetic training data using sliding-window segmentation on pre-existing set.
    '''
    window_segs = []
    for trans, gls in zip(transcriptions,glosses):
        transsplit = trans.split()
        glssplit = gls.split()
        for window in windows:
            if len(glssplit) < window:
                continue
            sid = 0
            for i in range(len(glssplit)-1):
                transwind = transsplit[sid:sid+window]
                glsswind = glssplit[sid:sid+window]
                if len(transwind) < window:
                    continue
                window_segs.append((orth_process(" ".join(transwind)),gloss_process(" ".join(glsswind))))

                sid+=1
    return window_segs
        

In [15]:
class LangIGT:
    '''
    Data from a single language, processed for sequence-to-sequence transduction.
    '''
    def __init__(self, lang):
        self.lang = lang
        self.train_ld, self.dev_ld, self.test_ld = self.get_langdict()
        self.windows = get_windows(self.train_ld["glosses"])
        self.train_splits = get_linesplits(self.train_ld["transcriptions"],self.train_ld["glosses"])
        self.dev_splits = get_linesplits(self.dev_ld["transcriptions"],self.dev_ld["glosses"])
        self.test_splits = get_linesplits(self.test_ld["transcriptions"],self.test_ld["glosses"])
        self.train_window_segs = get_window_segs(self.train_ld["transcriptions"],self.train_ld["glosses"], self.windows)
        self.gls_alphabet, self.orth_alphabet = self.get_alphabet()
        
    
    def get_langdict(self):
        ld = {
            "train": defaultdict(list),
            "dev": defaultdict(list),
            "test" : defaultdict(list)
        }
        path = PATH + "data/" + self.lang +"/"
        #path = f"data/{self.lang}/"

        for split in ["train", "dev", "test"]:
            fp = f"{lang2code[self.lang]}-{split}-track1-uncovered"
            with open(path + fp, "r", encoding="utf-8") as f:
                for line in f.readlines():
                    if line.startswith("\\t"):
                        ld[split]["transcriptions"].append(line.lstrip("\\t ").rstrip("\n"))
                    if line.startswith("\g"):
                        ld[split]["glosses"].append(line.lstrip("\\g ").rstrip("\n"))
                    if line.startswith("\l"):
                        ld[split]["translation"].append(line.lstrip("\\l ").rstrip("\n"))
        
        return ld["train"], ld["dev"], ld["test"]

    def get_alphabet(self):
      gloss_counter, orth_counter = Counter(), Counter()

      for gloss,trans in zip(self.train_ld["glosses"],self.train_ld["transcriptions"]):
        for g,t in zip(tag_bd(list(re.sub(" ", "#", gloss))), list(re.sub(" ", "#", trans))):
          gloss_counter.update([g])
          orth_counter.update([t])


      gloss_voc = vocab(OrderedDict(sorted(gloss_counter.items(), key=lambda x: x[1], reverse=True)), specials=SPECIALS)
      gloss_voc.set_default_index(gloss_voc[UNK])
      orth_voc = vocab(OrderedDict(sorted(orth_counter.items(), key=lambda x: x[1], reverse=True)), specials=SPECIALS)
      orth_voc.set_default_index(orth_voc[UNK])

      return gloss_voc, orth_voc

    def data(self,split):

      train = [i for i in self.train_splits] + [i for i in self.train_window_segs] # Augment just the training set
      data = {
            "train" : train,
            "dev" : self.dev_splits,
            "test" : self.test_splits
        }
      return data[split]

    def alphabet(self,split):
      alphabet = {
            "gloss" : self.gls_alphabet,
            "orth" : self.orth_alphabet
        }
      return alphabet[split]

In [16]:
# Initialize data for given language
CODE = "git" 
lang = LangIGT(code2lang[CODE]) 
MAXLEN = max([len(i) for _,i in lang.train_splits])

In [17]:
print(f"window sizes: {lang.windows}")
print(f"size of training set: {len(lang.data('train'))}")
print(f"size of dev set: {len(lang.data('dev'))}")
print(f"size of test set: {len(lang.data('test'))}")

print(f"longest gloss length in train set: {MAXLEN}")


window sizes: {1, 2, 3, 4}
size of training set: 858
size of dev set: 42
size of test set: 37
longest gloss length in train set: 120


# Tensor transformations for batched training



In [18]:
BATCH_SIZE = 32

def collate_batch(batch):
    input, output=[], []
    for morphemes, glosses in batch:
        input_tensor = torch.tensor([orth_alpha[i] for i in morphemes], dtype=torch.long)
        output_tensor = torch.tensor([gloss_alpha[i] for i in glosses], dtype=torch.long)
        input.append(input_tensor)
        output.append(output_tensor)

    return pad_sequence(input, 
                        batch_first=False, 
                        padding_value=orth_alpha[PAD]), pad_sequence(output, 
                        batch_first=False, 
                        padding_value=gloss_alpha[PAD])


def torchify(train,dev,test):    
    train_dataloader = DataLoader(
        train,
        batch_size=BATCH_SIZE,
        collate_fn=collate_batch,
        shuffle=True
    )
    dev_dataloader = DataLoader(
        dev,
        batch_size=BATCH_SIZE,
        collate_fn=collate_batch,
        shuffle=True
    )
    test_dataloader = DataLoader(
        test,
        batch_size=1,
        collate_fn=collate_batch,
        shuffle=False
    )
    return train_dataloader, dev_dataloader, test_dataloader


if LOAD_FROM_CKPT:

  train_dataloader = pickle.load(open(PATH + f"dataloaders/{CODE}_train_dataloader", "rb"))
  dev_dataloader = pickle.load(open(PATH + f"dataloaders/{CODE}_dev_dataloader", "rb"))
  test_dataloader = pickle.load(open(PATH + f"dataloaders/{CODE}_test_dataloader", "rb"))
  gloss_alpha = pickle.load(open(PATH + f"dataloaders/{CODE}_trg_alpha", "rb"))
  orth_alpha = pickle.load(open(PATH + f"dataloaders/{CODE}_src_alpha", "rb"))  

else:
  train, dev, test = lang.data("train"), lang.data("dev"), lang.data("test")
  gloss_alpha, orth_alpha = lang.alphabet("gloss"), lang.alphabet("orth")
  loader_path = PATH + "dataloaders/"

  train_dataloader, dev_dataloader, test_dataloader = torchify(train, dev, test)

  ## Save for reproducibility

  with open(PATH+f"dataloaders/{CODE}_train_dataloader", "wb") as f:
      pickle.dump(train_dataloader, f)

  with open(PATH+f"dataloaders/{CODE}_dev_dataloader", "wb") as f:
      pickle.dump(dev_dataloader, f)

  with open(PATH+f"dataloaders/{CODE}_test_dataloader", "wb") as f:
      pickle.dump(test_dataloader, f)

  with open(PATH+f"dataloaders/{CODE}_src_alpha", "wb") as f:
      pickle.dump(orth_alpha, f)

  with open(PATH+f"dataloaders/{CODE}_trg_alpha", "wb") as f:
      pickle.dump(gloss_alpha, f)

In [19]:
## Make sure data looks ok
for batch in train_dataloader:
    src, trg = batch
    print('source tensor size:', src.shape)
    print('target tensor size:', trg.shape)
    print('the tensor of first example in target:', trg[:, 0])
    print('the tensor of first example in src:', src[:, 0])
    print([orth_alpha.get_itos()[i] for i in src[:, 0]])
    print([gloss_alpha.get_itos()[i] for i in trg[:, 0]])
    break

source tensor size: torch.Size([40, 32])
target tensor size: torch.Size([42, 32])
the tensor of first example in target: tensor([ 1, 18, 18, 17, 26,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0])
the tensor of first example in src: tensor([ 1, 37, 17,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0])
['<start>', 'I', 'i', '<end>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<start>', 'C', 'C', 'N', 'J', '<end>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

# Model building 
## Architecture: PyTorch TransformerModel

In [20]:
## HYPERPARAMETERS
SRC_VOCAB_SIZE = len(orth_alpha)
TGT_VOCAB_SIZE = len(gloss_alpha)
EMB_SIZE = 300  
NHEAD = 6
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 50
DROPOUT = 0.25

In [21]:
SRC_VOCAB_SIZE

55

In [22]:

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = MAXLEN + 300):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, emb_size, embedding: Vectors):
        super(TokenEmbedding, self).__init__()
        #self.embedding = nn.Embedding(vocab_size, emb_size)
        self.embed = embedding
        #self.emb_size = emb_size
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embed(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 src_embeddings: nn.Embedding,
                 tgt_embeddings: Vectors,
                 dim_feedforward: int = 512,
                 dropout: float = DROPOUT):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(emb_size, src_embeddings)
        self.tgt_tok_emb = TokenEmbedding(emb_size, tgt_embeddings)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [23]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == orth_alpha.get_stoi()[PAD]).transpose(0, 1)
    tgt_padding_mask = (tgt == gloss_alpha.get_stoi()[PAD]).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

# Training and inference

## Initialize parameters

In [29]:
torch.manual_seed(531)

src_embeddings = nn.Embedding(SRC_VOCAB_SIZE, EMB_SIZE)
tgt_embeddings = nn.Embedding(TGT_VOCAB_SIZE, EMB_SIZE)


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, src_embeddings,
                                 tgt_embeddings, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [30]:
def train_epoch(model, iterator, optimizer):
    model.train()
    losses = 0

    for src, tgt in tqdm(iterator):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(iterator)


def evaluate(model, iterator):
    model.eval()
    losses = 0

    for src, tgt in iterator:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(iterator)

In [31]:
from timeit import default_timer as timer
import pandas as pd

def train_eval(NUM_EPOCHS, transformer, train_dataloader, optimizer, dev_dataloader, lang_code):
    losses = defaultdict(list)
    for epoch in range(1, NUM_EPOCHS + 1):
        prev_loss = 0
        start_time = timer()
        train_loss = train_epoch(transformer, train_dataloader, optimizer)
        end_time = timer()
        val_loss = evaluate(transformer, dev_dataloader)

        if epoch % 2 == 0:

          torch.save(transformer.state_dict(),
                     PATH + "models/" + lang_code + str(epoch))
          print(f"Checkpoint saved at epoch={epoch}")

        print((f"Epoch: {epoch},  "f"Epoch time = {(end_time - start_time):.3f}s"))
        print(f'\t Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {val_loss:.3f} |  Val. PPL: {math.exp(val_loss):7.3f}')

        losses["train_loss"].append(train_loss)
        losses["val_loss"].append(val_loss)
        losses_df = pd.DataFrame.from_dict(losses)
        losses_df.to_csv(PATH + "losses/" + lang_code +".csv")
    return losses


In [32]:
losses = train_eval(30, transformer, train_dataloader, optimizer, dev_dataloader, CODE)

100%|██████████| 27/27 [00:01<00:00, 25.67it/s]


Epoch: 1,  Epoch time = 1.057s
	 Train Loss: 4.401 | Train PPL:  81.506
	 Val. Loss: 3.981 |  Val. PPL:  53.594


  0%|          | 0/27 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 27/27 [00:00<00:00, 28.61it/s]


Checkpoint saved at epoch=2
Epoch: 2,  Epoch time = 0.950s
	 Train Loss: 3.933 | Train PPL:  51.034
	 Val. Loss: 3.539 |  Val. PPL:  34.423


  0%|          | 0/27 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 27/27 [00:00<00:00, 29.38it/s]


...

Checkpoint saved at epoch=28
Epoch: 28,  Epoch time = 0.984s
	 Train Loss: 0.984 | Train PPL:   2.674
	 Val. Loss: 3.002 |  Val. PPL:  20.126


  0%|          | 0/27 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 27/27 [00:00<00:00, 28.48it/s]


Epoch: 29,  Epoch time = 0.954s
	 Train Loss: 0.958 | Train PPL:   2.607
	 Val. Loss: 3.048 |  Val. PPL:  21.077


  0%|          | 0/27 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 27/27 [00:00<00:00, 27.40it/s]


Checkpoint saved at epoch=30
Epoch: 30,  Epoch time = 0.992s
	 Train Loss: 0.921 | Train PPL:   2.512
	 Val. Loss: 3.094 |  Val. PPL:  22.068


In [33]:
test_tensors = [src for src, _ in test_dataloader]
transformer_best = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, src_embeddings,
                                tgt_embeddings, FFN_HID_DIM)

ckpt = torch.load(PATH + f'models/{CODE}{22}',map_location=DEVICE)
transformer_best.load_state_dict(ckpt,strict=False)
transformer_best = transformer_best.to(DEVICE)

In [34]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len+10):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == gloss_alpha.get_stoi()[END]:
            break
    return ys

# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_tensor, extend_by=50):
    model.eval()
    src = src_tensor.view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  
        src, 
        src_mask, 
        max_len=num_tokens + extend_by, 
        start_symbol=orth_alpha.get_stoi()[START]
    ).flatten()

    return re.sub("<start>|<end>", "", ''.join([gloss_alpha.get_itos()[t] for t in tgt_tokens]))#" ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [38]:
PRED_PATH = "/content/drive/MyDrive/Colab_Notebooks/2023glossingST/final_preds/"
def get_predictions(transformer_best, test_tensors):
  test_tensors = [t.to(DEVICE) for t in test_tensors]
  
  preds = [translate(transformer_best, t) for t in tqdm(test_tensors)]
  preds = [p.replace("--","-").replace("@", " ??? ").replace("#", " ").replace("..",".") for p in preds]
  return preds

def write_preds(predictions,code,transl=True):

  ## WRITE GENERATED PREDICTIONS TO FILE IN EXPECTED FORMAT

  preds=[]
  transcriptions = lang.test_ld["transcriptions"]
  translations = lang.test_ld["translation"]
  for t, g, l in zip(transcriptions, predictions, translations):
    preds.append(f"\\t {t}")
    preds.append(f"\g {g}")
    preds.append(f"\l {l}")
    preds.append(" ")

  with open(f"{PRED_PATH}{code}-test-track1.txt", "w", encoding="utf-8") as fout:
    fout.write("\n".join(preds))


In [36]:
preds = get_predictions(transformer_best, test_tensors)

100%|██████████| 37/37 [00:20<00:00,  1.82it/s]
